Hannah Weiser, 3DGeo Group, Heidelberg University

October 2021

***

# Laser scanning simulation Arboretum Heidelberg

In June 2021, the 3DGeo group and students of Heidelberg university acquired TLS point clouds of the trees in the Arboretum in Heidelberg, which includes several Sequoias. Using this data, let’s create a ULS simulation! We can compare this simulated point cloud with a real ULS point cloud acquired over the same area in August 2021.

## Setup

We first have to make sure we have all requirements installed. Using Anaconda, please create the environment provided with the .yml file `pdal_pyhelios_env.yml` which you find in the helios root directory by typing: 

```
conda env create -f pdal_pyhelios_env.yml
```

## Imports

In [8]:
import pdal
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pathlib import Path, PurePath
import shutil
import time
import math
import laspy
import urllib.request
import sys
helios_path = str(Path.cwd().parent)
helios_run_path = str(PurePath(helios_path,'run/'))
sys.path.append(helios_path)
sys.path.append(helios_run_path)
from run import pyhelios
from pyheliostools import outputToNumpy
from extra.flight_planner import *

In [5]:
# Magic command to enable interactive plot.
%matplotlib widget

## Input data
`arboretum_data/2021-06-18_TLS_Arboretum_VZ-2000i_preprocessed.laz`

This is a preprocessed point cloud of the scene. We did a simple subsampling in [CloudCompare](www.cloudcompare.org) using the "space" option and a value of 10 cm. We then extracted a rectangular subsection of the entire point cloud (bounding box: 478845.830, 5471966.184, 478983.243, 5472121.949). We filtered this point cloud by deviation, keeping only points with values below 25. Lastly, we applied a statistical outlier removal (SOR) filter with the standard deviation quantified on 6 points and a threshold *nSigma* of 1.5.

Let's check if the file is already existing, other wise we will download it.

In [6]:
def reporthook(count, block_size, total_size):
    percent = min(int(count * block_size * 100 / total_size), 100)
    print("\r...{}%".format(percent), end="")
        
def download_data(filename, url):
    if not Path(filename).exists():
        print("Downloading data from '{}' to '{}'. Please wait ...".format(url, filename))
        if not Path(filename).parent.exists():
            Path(filename).parent.mkdir()
        urllib.request.urlretrieve(url, filename, reporthook=reporthook)
        print("\nDownload finished")
    else:
        print("File already exists. Great!")

data_tls = "../arboretum_data/2021-06-18_TLS_Arboretum_VZ-2000i_preprocessed.laz"
data_uls = "../arboretum_data/2021-08-17_ULS_Arboretum_AOI.laz"
url_tls = "https://3dweb.geog.uni-heidelberg.de/helios_example_arboretum/2021-06-18_TLS_Arboretum_VZ-2000i_preprocessed.laz"
url_uls = "https://3dweb.geog.uni-heidelberg.de//helios_example_arboretum/2021-08-17_ULS_Arboretum_AOI.laz"

download_data(data_tls, url_tls)
download_data(data_uls, url_uls)

File already exists. Great!
File already exists. Great!


## Point cloud classification and DTM computation

Next, we compute a digital terrain model (DTM), which we will model in HELIOS++ separately from the trees. We use the [python extension](https://github.com/PDAL/python) of the open source library [PDAL](https://pdal.io/) (Point Data Abstraction Library) to create the DTM.

First, we define our output filenames.

In [ ]:
data_classified = data_tls.replace("preprocessed", "classified")
dtm_output = data_tls.replace("preprocessed.laz", "dtm_50cm.tif")
vegetation_output = data_tls.replace("preprocessed.laz", "vegetation.xyz")

We start by classifying the point cloud into ground and non-ground points using the Simple Morphological Filter (SMRF) (see also [Pingel et al. 2013](https://doi.org/10.1016/j.isprsjprs.2012.12.002)).

In [ ]:
json_classification = """[
    "%s",
    {
        "type":"filters.smrf",
        "ignore":"Classification[7:7]",
        "window":33,
        "slope":0.2,
        "threshold":0.5,
        "cell":1.0
    },
    {
        "type":"filters.outlier",
        "method":"statistical",
        "mean_k":8,
        "multiplier":3.0
    },
    {
        "type":"writers.las",
        "filename": "%s",
        "compression":"true"
    }
]"""

In [ ]:
pipeline = pdal.Pipeline(json_classification % (data_tls, data_classified))
count = pipeline.execute()

Then, we use only the ground points (class 2, according to the ASPRS Standard point classes of the [LAS format specification](http://www.asprs.org/wp-content/uploads/2019/07/LAS_1_4_r15.pdf), p. 19, Table 9) to create a DTM of 1 m resolution.

In [ ]:
json_dtm = """[
    "%s",
    {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    },
    {
        "type":"writers.gdal",
        "filename": "%s",
        "output_type":"min",
        "gdaldriver":"GTiff",
        "resolution":1.0,
        "window_size":8
    }
]"""

In [ ]:
pipeline = pdal.Pipeline(json_dtm % (data_classified, dtm_output))
count = pipeline.execute()

We also want to export the non-ground points (= vegetation points) to an ASCII format. 

In [ ]:
json_vegetation = """[
    "%s",
    {
        "type":"filters.range",
        "limits":"Classification[0:1]"
    },
    {
        "type":"writers.text",
        "filename": "%s",
        "order":"X,Y,Z",
        "precision":6,
        "keep_unspecified":"false",
        "write_header":"false"
    }
]"""

In [ ]:
pipeline = pdal.Pipeline(json_vegetation % (data_classified, vegetation_output))
count = pipeline.execute()

Now we have all the **scenepart** files needed to build a survey: The DTM and the vegetation point cloud.

## Writing the scene

To perform the scene generation and flight planning, we will use the extra tool `flight_planner.py` which comes with HELIOS++. You can find it in the folder `extra` and import it with `from extra.flight_planner import *`. 

Let's first move our **scenepart** files into a folder "arboretum_hd" in "helios\data\sceneparts":

In [ ]:
scenepart_folder = Path("../data/sceneparts/arboretum_hd")
if not scenepart_folder.exists():
    Path.mkdir(scenepart_folder)

In [ ]:
dtm_file = PurePath(scenepart_folder, PurePath(dtm_output).name)
shutil.copy(dtm_output, dtm_file)

In [ ]:
vegetation_file = PurePath(scenepart_folder, PurePath(vegetation_output).name)
shutil.copy(vegetation_output, vegetation_file)

Next, we will assemble the **scene**. In the scene-folder (`helios\data\scenes`), let’s create a new folder which we will also call `arboretum_hd`. Here, we will create a new xml-file called `arboretum_VZ-2000i.xml`. 

In [ ]:
scene_folder = Path("../data/scenes/arboretum_hd")
if not scene_folder.exists():
    Path.mkdir(scene_folder)

In [ ]:
dtm_part = create_scenepart_tiff(dtm_file)
vegetation_part = create_scenepart_xyz(vegetation_file, sep=",", voxel_size=0.1)
scene_id = "arboretum"
scene_content = build_scene(id=scene_id, name="Arboretum", sceneparts=[dtm_part, vegetation_part])

In [ ]:
print(scene_content)

The first `<part>` is the DTM, which is loaded with the `geotiffloader`.

The second `<part>` is the vegetation, for which we voxelize the vegetation point cloud with a voxel size of 0.2 m using the `xyzloader`.

In [ ]:
scene_file = scene_folder / "arboretum_VZ-2000i.xml"

In [ ]:
with open(scene_file, "w") as f:
    f.write(scene_content)

## Configuring the survey

Now we have to configure the survey. We create another XML fille in the `surveys` folder (`helios/data/surveys`), which we may call `arboretum_uls.xml`.

In [ ]:
survey_file = "../data/surveys/arboretum_uls.xml"

To perform the flight planning, we will also use `flight_planner.py`.

We first define the bounding box of our area of interest.

In [ ]:
bbox = [478845.830, 5471966.184, 478983.243, 5472121.949]

Next, we create a flight plan for this bounding box for a given strip spacing, and flight pattern. We furthermore rotate this pattern by 45 degrees.

In [ ]:
wp, c, dist = compute_flight_lines(bbox, spacing=30, rotate_deg=45, flight_pattern="criss-cross")

Let's visualize our flight plan and validate that it looks as expected.

In [ ]:
plot = plot_flight_plan(wp)

Now, we'll set the flying speed and the altitude.

In [ ]:
speed = 5.0
altitude = 465  # important: Absolute height, not height above ground!

And export the legs for the survey XML-file.

In [ ]:
legs = export_for_xml(wp, altitude=altitude, template_id="uls_template", speed=speed)

We can now write our survey file:

In [ ]:
survey_content = """<?xml version="1.0" encoding="UTF-8"?>
<document>
	<scannerSettings id="uls_template" active="true" scanAngle_deg="90" pulseFreq_hz="100000" scanFreq_hz="50" />
	<survey name="Arboretum" platform="data/platforms.xml#copter_linearpath" scanner="data/scanners_als.xml#riegl_vux-1uav" scene="{}#arboretum">
    {}
	</survey>
</document>

""".format(scene_file, legs)

We define the default scanner settings in the first tag (`<scannerSettings ...>`). In the `<survey ...>` tag, we provide the scanner, platform, and scene that the survey should use. Then we need to write the individual legs between the opening and the closing survey tags to define the scanner location, movement and speed. 

We can now write our survey to the file:

In [ ]:
with open(survey_file, "w") as f:
    f.write(survey_content)

Now we are ready to go. We can execute the survey from this notebook using the helios python bindings that we imported in the beginning of the notebook.

Configure simulation context and build a simulation:

In [ ]:
# Sim context.
# Set logging.
pyhelios.loggingQuiet()
# Set seed for random number generator.
pyhelios.setDefaultRandomnessGeneratorSeed("123")

In [ ]:
# Build simulation parameters: (surveyPath, assetsPath, outputPath, ...).
sim = pyhelios.Simulation(
    str(survey_file),
    'assets/',
    'output/',
    0,          # Num Threads
    True,      # LAS output
    False,     # LAS1.0 output
    True,      # ZIP output
    )

# Enable final output.
sim.finalOutput = True

# Set sim frequency.
sim.simFrequency = 10000

Load survey file:

In [ ]:
# Load survey file. Further configuration of survey possible.
sim.loadSurvey(
        True,       # Leg Noise Disabled FLAG
        False,      # Rebuild Scene FLAG
        False,      # Write Waveform FLAG
        False,      # Calc Echowidth FLAG
        False,      # Full Wave Noise FLAG
        True        # Platform Noise Disabled FLAG
            )

In [ ]:
# Start the simulation.
start_time = time.time()
sim.start()

if sim.isStarted():
    print('Simulation has started!\nSurvey Name: {survey_name}\n{scanner_info}'.format(
        survey_name = sim.getSurvey().name,
        scanner_info = sim.getScanner().toString()))

In [ ]:
while sim.isRunning():
    duration = time.time()-start_time
    mins = duration // 60
    secs = duration % 60
    print("\r"+"Simulation is running since {:d} min and {:d} sec. Please wait.".format(mins, secs), end="")
    time.sleep(10)

if sim.isFinished():
    print("\n"+"Simulation has finished!")

Once the simulation is finished, we can process the outputs.

In [ ]:
# Create instance of PyHeliosOutputWrapper class using sim.join(). 
# Contains attributes 'measurements' and 'trajectories' which are Python wrappers of classes that contain the output vectors.
output = sim.join()

# Create instances of vector classes by accessing 'measurements' and 'trajectories' attributes of output wrapper.
measurements = output.measurements
trajectories = output.trajectories

# Get amount of points in trajectory and amount of measurements by accessing length of measurement and trajectory vectors.
print('Number of measurements : {n}'.format(n=len(measurements)))
print('Number of points in trajectory : {n}'.format(n=len(trajectories)))

The simulation outputs can be found in `helios\output\Survey Playback\Arboretum\YYYY-MM-DD_HH-MM-SS\points`. For each run a folder will be created in the "Arboretum"-folder named by the timestamp of execution. There is one point cloud and trajecotry file for each flight strip.

## Plot the point cloud and trajectory

To visualize the output, we can make use of **pyheliostools**, a library of functions associated with pyhelios. It should be included in your helios package. Specifically, we will use the function **outputToNumpy**, which saves both measurement and trajectory data as numpy Arrays We already imported it in the beginning of the notebook.

In [ ]:
# Output is two np.arrays. One for measurements and one for trajectory points.
measurement_points, trajectory_points = outputToNumpy(output)

We saw earlier, that we simulated more than 9 million points. Let's subsample them for plotting and show only every 50th point. 

In [ ]:
# Points to be plotted:
# First three cols are x, y and z vals
traj_points=trajectory_points[:,:3]
points = measurement_points[::50,:3]

In [ ]:
def set_axes_equal(ax):
    '''Make axes of 3D plot have equal scale so that spheres appear as spheres,
    cubes as cubes, etc..  This is one possible solution to Matplotlib's
    ax.set_aspect('equal') and ax.axis('equal') not working for 3D.

    Input
      ax: a matplotlib axis, e.g., as output from plt.gca().
    '''

    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()

    x_range = abs(x_limits[1] - x_limits[0])
    x_middle = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_middle = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_middle = np.mean(z_limits)

    # The plot bounding box is a sphere in the sense of the infinity
    # norm, hence I call half the max range the plot radius.
    plot_radius = 0.5*max([x_range, y_range, z_range])

    ax.set_xlim3d([x_middle - plot_radius, x_middle + plot_radius])
    ax.set_ylim3d([y_middle - plot_radius, y_middle + plot_radius])
    ax.set_zlim3d([z_middle - plot_radius, z_middle + plot_radius])

In [ ]:
# Matplotlib figure.
fig = plt.figure()
# Axes3d axis onto mpl figure.
ax = fig.add_subplot(projection='3d')

# Scatter plot of points (coloured by height).
ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=points[:, 2], cmap="rainbow", s=0.02, label='scene')
# Plot of trajectory.
ax.plot(traj_points[:,0], traj_points[:,1], traj_points[:,2], c = 'black', label = 'scanner trajectory')

# Add axis labels.
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.set_zlabel('$Z$')

set_axes_equal(ax)

# Set title.
ax.set_title(label='Point cloud and trajectory from pyhelios simulation in Heidelberg')
# Set subtitle.
ax.text2D(0.1, 0.97, "survey: {s}".format(s=sim.getSurvey().name, n=len(trajectories)),
                                                                              fontsize='8', transform=ax.transAxes)
ax.view_init(elev=10, azim=145)

# Display results
plt.show()

Let's extract a section of the original and the simulated point cloud...

In [ ]:
xmin, ymin, xmax, ymax = [478894.3, 5472000.0, 478899.3, 5472100.0]

section_sim = measurement_points[(measurement_points[:, 0] > xmin) & 
                                 (measurement_points[:, 0] < xmax) & 
                                 (measurement_points[:, 1] > ymin) & 
                                 (measurement_points[:, 1] < ymax)]
import laspy
with laspy.open(data_uls) as f:
    las = f.read()

    arr = np.vstack((las.x, las.y, las.z)).transpose()
    section_or = arr[(arr[:, 0] > xmin) & 
                     (arr[:, 0] < xmax) & 
                     (arr[:, 1] > ymin) & 
                     (arr[:, 1] < ymax)]

... and plot this:

In [ ]:
# Matplotlib figure.
fig = plt.figure(figsize=(9,5))
# Axes3d axis onto mpl figure.
ax = fig.add_subplot()

# Scatter plot of original and simulated points in different colors
ax.scatter(section_or[:, 1], section_or[:, 2], c="orange", s=0.01)
ax.scatter(section_sim[:, 1], section_sim[:, 2], c="cornflowerblue", s=0.01)

# Add axis labels.
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.axis('equal')

# Set title.
from textwrap import wrap
title = ax.set_title("\n".join(wrap("Point cloud of a section through the original and simulated ULS point clouds.", 50)))

plt.show()

Let's create another plot where we can see the point clouds side by side.

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(8,12))
axs[0].scatter(section_or[:, 1], section_or[:, 2], color="orange", s=0.01)
axs[0].set_title("Original point cloud")
axs[0].axis("equal")
#plt.subplot(212)
axs[1].scatter(section_sim[:, 1], section_sim[:, 2], color="cornflowerblue", s=0.01)
axs[1].set_title("Simulated point cloud")
axs[1].axis("equal")
fig.subplots_adjust(hspace=0.2)
plt.show()

We can see some differences in the simulated point clouds, especially the upper canopy. In the input TLS point clouds, there are occluded areas and areas with low point density in the upper canopy. This will result in an "incomplete" or sparse voxel model of some tree crowns, which of course propagates into the simulated point cloud.